In [3]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from sklearn_pandas import DataFrameMapper
from sklearn import preprocessing
import itertools

# Display more rows and get rid of the margins
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Display mulitiple values from each cell
# https://stackoverflow.com/a/42476224
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



# Define paths and functions

In [4]:
# Define the directories that we're going to read data from
data_dir = Path('data')
swarm_dir = Path('swarm_dir')

In [5]:
def bal_samp(df, strata, balance, order, keys, n_splits=5, n_draws=100):
    """Balanced sampling across strata
    
    Parameters
    ----------
    df: pandas.DataFrame
        Dataframe from which you want to sample
    strata: str or list of str
        Name(s) of the column or columns that define the groups
        from which you want a balanced sample
    balance: str or list of str
        Name(s) of the columns or columns containing the factors
        you want to evenly sample across strata
    order: str or list of str
        Name(s) of the column whose distribution you want to preserve
    keys: list of str
        Name(s) of the column(s) that you will use to match
        the output back to your original column
    n_splits: int
        Number of cross validation folds you want to create per draw
    n_draws: int
        Number of balanced samples of your dataset you want to create
    
    Returns
    -------
    draws_df: pandas.DataFrame
        Dataframe with number of rows about equal to number of rows in df
        and number of columns equal to n_draws + len(keys) + len(strata) + len(balance) + len(order).
        Contains the crossfold labels for balanced sampling across the strata you've defined.
    """
    # create dict of minimum count in each strata of each combination of balance factor
    bal_dict = (df.groupby(strata + balance)[[keys[0]]]
                  .nunique()
                  .groupby(balance)
                  .min()
                  .to_dict('index'))
    bal_dict = {k:v[keys[0]] for k,v in bal_dict.items()}
    
    # Appologies for the disgusting nested loops
    # For each draw, at each strata level, for each unique combin
    draws_df = []
    # For each draw
    for nn in range(n_draws):
        strat_df = []
        # From each strata group
        for x, gbdf in df.groupby(strata):
            cvs_df = []
            # from each unique combination of balance values
            for bal_vals, num in bal_dict.items():
                # create an index selecting the rows at those balance values
                ind = np.ones((len(gbdf))).astype(bool)
                for bcol,bv in zip(balance,bal_vals):
                    ind = np.logical_and(ind, gbdf[bcol] == bv)
                # draw a random sample of the group members 
                # that meet the balance criteria
                # and sort them by the order values
                bal_df = gbdf[ind].sample(n=num).sort_values(order).loc[:,keys]
                # create a list of the cross validation values long enough to match
                cv_inds = list(np.arange(n_splits))*((len(bal_df)//n_splits)+1)
                bal_df['draw_%d'%nn] = cv_inds[:len(bal_df)]
                # and append them to a list
                cvs_df.append(bal_df)
            # combine these lists to get all the rows for that strata
            # and append them to create a list of selected rows from all the strata
            strat_df.append(pd.concat(cvs_df).loc[:,['draw_%d'%nn]])
        # pull these all together to create the draws dataframe
        draws_df.append(pd.concat(strat_df))
    draws_df = pd.concat(draws_df, axis=1)
    # Merge back in the indicator variables
    draws_df = (df.loc[:,
                      keys 
                      + strata 
                      + balance 
                      + order]
                  .merge(draws_df,
                         right_index=True,
                         left_index=True,
                         how='left')
               )
    # make sure the shape is still ok
    assert draws_df.shape[0] == df.shape[0]
    assert draws_df.shape[1] == (n_draws
                                 + len(keys)
                                 + len(strata)
                                 + len(balance)
                                 + len(order))
    return draws_df


def gen_binned_perms(df, bin_levels, n_perms = 1000):
    df = df.copy(deep=True)
    permed_inds = []
    permed_inds.append(df.index.values)
    if not 'ind' in df.columns:
        df['ind'] = df.index.values
    else:
        raise Exception
    for pn in range(n_perms):
        permed_inds.append(df.groupby(bin_levels).ind.transform(np.random.permutation).values)
    return permed_inds


def get_cols(raw_df):
    base_meta_cols = ['contrast', 'fmri_beta_gparc_numtrs', 'fmri_beta_gparc_tr', 'lmt_run', 
                      'mid_beta_seg_dof', 'task', 'collection_id', 'dataset_id', 'subjectkey',
                      'src_subject_id', 'interview_date', 'interview_age',
                      'gender', 'event_name', 'visit', 'rsfm_tr', 'eventname',
                      'rsfm_nreps', 'rsfm_numtrs', 'pipeline_version',  'scanner_manufacturer_pd',
                      'scanner_type_pd','deviceserialnumber', 'magnetic_field_strength',
                      'procdate', 'collection_title', 'promoted_subjectkey', 'study_cohort_name','ehi_y_ss_scoreb', '_merge', 'qc_ok', 'age_3mos']
    meta_cols = raw_df.columns[raw_df.columns.isin(base_meta_cols)].values
    metric_cols = raw_df.columns[~raw_df.columns.isin(base_meta_cols)].values
    return metric_cols, meta_cols


def big_sites(size_limit, df, metric_cols):
    notnull_mask = np.logical_and(~(df.loc[:,metric_cols].isnull().sum(1).astype(bool)),
                                  np.isfinite(df.loc[:,metric_cols]).sum(1).astype(bool))
    notnull_mask = np.logical_and(notnull_mask, df.deviceserialnumber.notnull())
    notnull_mask = np.logical_and(notnull_mask, df.qc_ok == 1)
    # for right now, just focus on sites with more than size_limit scans
    scans_per_sn = df.loc[notnull_mask,:].groupby('deviceserialnumber')[['collection_id']].count().sort_values('collection_id', ascending = False)
    big_sns = df.loc[df.deviceserialnumber.isin(scans_per_sn.query('collection_id > @size_limit').index)
                          & notnull_mask, :]
    scans_per_bigsn = big_sns.groupby('deviceserialnumber')[['collection_id']].count().sort_values('collection_id', ascending = False)
    
    print(f"Number of sites with more than {size_limit} scans:",len(big_sns.deviceserialnumber.unique()))
    print(f"Number of scans collected at sites with more than {size_limit} scans:",len(big_sns))
    scans_per_bigsn.query('collection_id > @size_limit')
    return big_sns


def drop_bad(df):
    notnull_mask = np.logical_and(~(df.loc[:,metric_cols].isnull().sum(1).astype(bool)),
                                  np.isfinite(df.loc[:,metric_cols]).sum(1).astype(bool))
    notnull_mask = np.logical_and(notnull_mask, df.deviceserialnumber.notnull())
    notnull_mask = np.logical_and(notnull_mask, df.qc_ok == 1)
    notnull_mask = np.logical_and(notnull_mask, (np.abs(df.loc[:,metric_cols]) < 100).product(1).astype(bool))
    return df.loc[notnull_mask,:].copy(deep=True)

# Prep Dataframes for swarm jobs


## rs-fMRI data

In [6]:
# Process connectivity data
# If processed connectivity data doesn't exist, create it
big_con_file = swarm_dir/'big_con.pkz'
if not big_con_file.exists():
    print("No processed connectivity data file found. Generating a new one.")
    con = pd.read_pickle(data_dir/'con.pkz')
    con_metric_cols, con_meta_cols = get_cols(con)
    big_con = big_sites(100, con, con_metric_cols)

    big_con.to_pickle(big_con_file)
else:
    big_con = pd.read_pickle(big_con_file)
    
# Look at counts in connectivity data    
big_con.groupby(['scanner_manufacturer_pd',
                 'scanner_type_pd',
                 'deviceserialnumber']).count()
    
# Load permutations or generate them
perm_file = swarm_dir/'perms_con.pkz'
if not perm_file.exists():
    print("No connectivity permutation file found. Generating a new one.")
    # within bucket shuffle
    big_con['age_3mos'] = (big_con['interview_age'] // 3) * 3
    bin_levels = ['gender', 'ehi_y_ss_scoreb', 'age_3mos']
    con_perms = gen_binned_perms(big_con, bin_levels, n_perms=2000)
else:
    with open(perm_file.as_posix(), 'rb') as h:
        perms = pickle.load(h)

# If ymapper doesn't exist, create it
# We're doing this outside the swarm to ensure that every worker has the same
# encoding of the categories
ymap_file = swarm_dir/'yfit_con.pkz'
if not ymap_file.exists():
    print("No connectivity ymapper file found. Generating a new one.")

    ymapper = DataFrameMapper([("deviceserialnumber",preprocessing.LabelEncoder())])
    yfit = ymapper.fit(big_con)
    with open(swarm_dir/'yfit_con.pkz', 'wb') as h:
        pickle.dump(yfit, h)

collection_id  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber                  
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05                  144   
                                        HASHc3bf3d9c                  177   
                                        HASHd7cb4c6d                  154   
Philips Medical Systems Achieva dStream HASHdb2589d4                  141   
                        Ingenia         HASH5ac2b20b                  139   
SIEMENS                 Prisma          HASH11ad4ed5                  229   
                                        HASH3935c89e                  309   
                                        HASH5b0cf1bb                  215   
                        Prisma_fit      HASH1314a204                  177   
                                        HASH7911780b                  102   
                                        HASH96a0c182                  115   
                                        HASHb640a1b8                  137   
                                        HASHc9398971                  120   

                                                            dataset_id  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber               
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05               144   
                                        HASHc3bf3d9c               177   
                                        HASHd7cb4c6d               154   
Philips Medical Systems Achieva dStream HASHdb2589d4               141   
                        Ingenia         HASH5ac2b20b               139   
SIEMENS                 Prisma          HASH11ad4ed5               229   
                                        HASH3935c89e               309   
                                        HASH5b0cf1bb               215   
                        Prisma_fit      HASH1314a204               177   
                                        HASH7911780b               102   
                                        HASH96a0c182               115   
                                        HASHb640a1b8               137   
                                        HASHc9398971               120   

                                                            subjectkey  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber               
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05               144   
                                        HASHc3bf3d9c               177   
                                        HASHd7cb4c6d               154   
Philips Medical Systems Achieva dStream HASHdb2589d4               141   
                        Ingenia         HASH5ac2b20b               139   
SIEMENS                 Prisma          HASH11ad4ed5               229   
                                        HASH3935c89e               309   
                                        HASH5b0cf1bb               215   
                        Prisma_fit      HASH1314a204               177   
                                        HASH7911780b               102   
                                        HASH96a0c182               115   
                                        HASHb640a1b8               137   
                                        HASHc9398971               120   

                                                            src_subject_id  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber                   
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05                   144   
                                        HASHc3bf3d9c                   177   
                                        HASHd7cb4c6d                   154   
Philips Medical Systems Achieva dStream HASHdb2589d4                   141   
                        Ingenia         HASH5ac2b20b                   139   
SIEMENS                 Prisma          HASH11ad4ed5                   229   
                                        HASH3935c89e      

We'll be generating these folds anew for each permutation to make sure that all the CV folds in the perms have the same counts in them, but I'm running htis code here so that I can pull out information about the draws.

In [8]:
# Create cv folds for gender age balance
strata = ['deviceserialnumber']
balance = ['gender', 'ehi_y_ss_scoreb']
order = ['interview_age']
keys = ['subjectkey', 'interview_date']
n_draws = 25
n_splits = 3
ab_n_splits = 2
draws_df = bal_samp(big_con, strata, balance, order, keys, n_splits=n_splits, n_draws=n_draws)

# Prep data for age_balance
strata = ['deviceserialnumber']
balance = ['gender', 'ehi_y_ss_scoreb','age_3mos']
order = ['interview_age']
keys = ['subjectkey', 'interview_date']
df_ab = big_con.copy(deep=True)
df_ab['age_3mos'] = (df_ab['interview_age'] // 3) * 3

count_by_level = df_ab.groupby(strata+balance)[['subjectkey']].nunique().reset_index()
levels_counts = (count_by_level[count_by_level.subjectkey >= ab_n_splits]
                 .groupby(balance)[strata]
                 .nunique()
                 .reset_index()
                 .rename(columns={'deviceserialnumber':'sufficient_sites'}))

# Look at the categories of subjects that have enough data per site
print("These are the categories of subjects with enough data per site.")
print("Enough defined as at least %d subjects in that category per site"%ab_n_splits)
levels_counts.loc[levels_counts.sufficient_sites == levels_counts.sufficient_sites.max()]

df_ab = df_ab.reset_index().merge(levels_counts, how='left', on=['gender','ehi_y_ss_scoreb', 'age_3mos']).set_index('index')
df_ab = df_ab.loc[df_ab.sufficient_sites == df_ab.sufficient_sites.max(),:]
ab_draws_df = bal_samp(df_ab, strata, balance, order, keys, n_splits=ab_n_splits, n_draws=n_draws)

These are the categories of subjects with enough data per site.
Enough defined as at least 2 subjects in that category per site


,gender,ehi_y_ss_scoreb,age_3mos,sufficient_sites
3,F,1,114,13
4,F,1,117,13
5,F,1,120,13
6,F,1,123,13
8,F,1,129,13
30,M,1,117,13
33,M,1,126,13
34,M,1,129,13


## task-based data

In [16]:
tb_b = pd.read_pickle(data_dir/'task_based_betas.pkz')
metric_cols, meta_cols = get_cols(tb_b)

# For task based analysis limit to only subjects that have data across all tasks and runs
tb_b_nb = drop_bad(tb_b)
tb_b_nb['cr'] = tb_b_nb.contrast + tb_b_nb.lmt_run
full_data_subs = pd.unique((tb_b_nb.groupby('subjectkey')[['cr']].nunique() == tb_b_nb.groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values)
tb_b_full = tb_b.loc[tb_b.subjectkey.isin(full_data_subs), :]

big_crs = []
for x,df in tb_b_full.query('lmt_run != "AVERAGE"').groupby(['contrast','lmt_run']):
    tmp = big_sites(100, df, metric_cols).sort_values('subjectkey').reset_index()
    assert len(tmp) > 0
    big_crs.append(tmp)
big_crs[-1].groupby(['scanner_manufacturer_pd',
                      'scanner_type_pd','deviceserialnumber']).count()
assert np.array([(big_crs[0].subjectkey == bc.subjectkey).all() for bc in big_crs]).all()

for crt in big_crs:
    assert (crt.index == big_crs[0].index).all()
    task = crt.task.unique()[0]
    run = crt.lmt_run.unique()[0]
    contrast = '-'.join(crt.contrast.unique()[0].split())
    crt_file = swarm_dir/f'crt_{task}_{contrast}_{run}_beta.pkz'
    if not crt_file.exists():
        crt.to_pickle(crt_file)

# Load permutations or generate them
crt_perm_file = swarm_dir/'perms_crt_beta.pkz'
if not crt_perm_file.exists():
    print("No task based permutation file found. Generating a new one.")
    # within bucket shuffle
    big_crs[0]['age_3mos'] = (big_crs[0]['interview_age'] // 3) * 3
    bin_levels = ['gender', 'ehi_y_ss_scoreb', 'age_3mos']
    crt_perms = gen_binned_perms(big_crs[0], bin_levels, n_perms=2000)
    with open(crt_perm_file, 'wb') as h:
        pickle.dump(crt_perms, h)
else:
    with open(crt_perm_file.as_posix(), 'rb') as h:
        crt_perms = pickle.load(h)

# If ymapper doesn't exist, create it
# We're doing this outside the swarm to ensure that every worker has the same
# encoding of the categories
crt_ymap_file = swarm_dir/'yfit_crt_beta.pkz'
if not crt_ymap_file.exists():
    crt_ymapper = DataFrameMapper([("deviceserialnumber",preprocessing.LabelEncoder())])
    crt_yfit = crt_ymapper.fit(big_crs[0])
    with open(crt_ymap_file, 'wb') as h:
        pickle.dump(crt_yfit, h)
else:
    with open(crt_ymap_file, 'rb') as h:
        crt_yfit = pickle.load(h)


Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of sites with more than 100 scans: 7
Number of scans collected at sites with more than 100 scans: 1065
Number of 

index  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber          
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05          109   
                                        HASHc3bf3d9c          130   
SIEMENS                 Prisma          HASH11ad4ed5          149   
                                        HASH3935c89e          275   
                                        HASH5b0cf1bb          145   
                        Prisma_fit      HASH1314a204          154   
                                        HASH96a0c182          103   

                                                            collection_id  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber                  
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05                  109   
                                        HASHc3bf3d9c                  130   
SIEMENS                 Prisma          HASH11ad4ed5                  149   
                                        HASH3935c89e                  275   
                                        HASH5b0cf1bb                  145   
                        Prisma_fit      HASH1314a204                  154   
                                        HASH96a0c182                  103   

                                                            collection_title  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber                     
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05                     109   
                                        HASHc3bf3d9c                     130   
SIEMENS                 Prisma          HASH11ad4ed5                     149   
                                        HASH3935c89e                     275   
                                        HASH5b0cf1bb                     145   
                        Prisma_fit      HASH1314a204                     154   
                                        HASH96a0c182                     103   

                                                            contrast  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber             
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05             109   
                                        HASHc3bf3d9c             130   
SIEMENS                 Prisma          HASH11ad4ed5             149   
                                        HASH3935c89e             275   
                                        HASH5b0cf1bb             145   
                        Prisma_fit      HASH1314a204             154   
                                        HASH96a0c182             103   

                                                            dataset_id  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber               
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05               109   
                                        HASHc3bf3d9c               130   
SIEMENS                 Prisma          HASH11ad4ed5               149   
                                        HASH3935c89e               275   
                                        HASH5b0cf1bb               145   
                        Prisma_fit      HASH1314a204               154   
                                        HASH96a0c182               103   

                                                            eventname  \
scanner_manufacturer_pd scanner_type_pd deviceserialnumber              
GE MEDICAL SYSTEMS      DISCOVERY MR750 HASH4b0b8b05              109   
                                        HASHc3bf3d9c              130   
SIEMENS                 Prisma          HASH11ad4ed5              149   
                                        HASH3935c89e              275   
                                        HASH5b0cf1bb              145   
                        Prisma_fit      HASH1314a204              154   
                                        HASH96a0c182              103   

                                                 

# Write swarm files

## rs-fMRI data

In [6]:
# TODO: Clean up paths here
from pathlib import Path
swarm_file = Path('swarm_con')
cmds = []
for pn in range(0,1001):

    out_path = '/lscratch/$SLURM_JOBID/pn-%04d_con.pkz'%(pn)
    rsync_in = 'rsync -avch /data/nielsond/abcd/abcd_meeting/swarm_dir /lscratch/$SLURM_JOBID/; '
    cmd_base = ("singularity exec  -H ~/temp_for_singularity /data/MLDSST/singularity_images/abcd_tmp-2018-04-26-a7f5b7d6a3b4.img "
                + f"python /lscratch/$SLURM_JOBID/swarm_dir/run_abcd_perm_new_draws.py {pn} /lscratch/$SLURM_JOBID/swarm_dir/perms_con.pkz "
                + f"/lscratch/$SLURM_JOBID/swarm_dir/yfit_con.pkz /lscratch/$SLURM_JOBID/swarm_dir/big_con.pkz "
                + f"{out_path} 30 --n_draws={n_draws} --do_ab; ")
    rsync_out = f'rsync -avch {out_path} /data/nielsond/abcd/abcd_meeting/swarm_out_cb_fix_new_draws/pn-%04d_con.pkz; '%(pn)
    cmd = rsync_in + cmd_base + rsync_out
    cmds.append(cmd)

cmds = '\n'.join(cmds)
swarm_file.write_text(cmds)

In [ ]:
# Then, outside of jupyter, run the swarm file:
# swarm -f swarm_test3 -g 64 -t 32 --logdir swarm_log --time 12:00:00 --partition=nimh,norm --gres=lscratch:20

In [7]:
singularity exec  -H ~/temp_for_singularity /data/MLDSST/singularity_images/abcd_tmp-2018-04-26-a7f5b7d6a3b4.img 

2001

## Task-based data

In [106]:
# TODO: Clean up paths here
from pathlib import Path
swarm_file = Path('swarm_test_crt')
n_draws = 25
bundle = 5
prange = (666,1001)
user='adamt'
                  
def make_cmd_list(user, prange, bundle = 5):
    cmds = []
    job_n = 0
    rsync_in = ''#'rsync -avch /data/MLDSST/abcd/abcd_swarm_dir /lscratch/$SLURM_JOBID/; '
    cmd = rsync_in
    for pn in range(*prange):
        for crt in big_crs:
            task = crt.task.unique()[0]
            run = crt.lmt_run.unique()[0]
            contrast = '-'.join(crt.contrast.unique()[0].split())

            out_path = f'/data/MLDSST/abcd/abcd_swarm_out/pn-%04d_crt_{task}_{contrast}_{run}.pkz'%(pn)
            #local_path = f'/data/MLDSST/abcd/abcd_swarm_out/pn-%04d_crt_{task}_{contrast}_{run}.pkz'%(pn)
            
            if not Path(out_path).exists():

                cmd_base = ('export SINGULARITY_BINDPATH="/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data"; singularity exec -H ~/temp_for_singularity /data/MLDSST/singularity_images/abcd_tmp-2018-04-26-a7f5b7d6a3b4.img '
                            + f"python /data/MLDSST/abcd/abcd_swarm_dir/run_abcd_perm_new_draws.py {pn} /data/MLDSST/abcd/abcd_swarm_dir/perms_crt_beta.pkz "
                            + f"/data/MLDSST/abcd/abcd_swarm_dir/yfit_crt_beta.pkz /data/MLDSST/abcd/abcd_swarm_dir/crt_{task}_{contrast}_{run}_beta.pkz "
                            + f"{out_path} 30 --n_draws={n_draws}; ")
                #rsync_out = f'rsync -ach {out_path} {local_path}; chown $USER:MLDSST {local_path}; '
                cmd +=  cmd_base #+ rsync_out
                job_n +=1
                if job_n == bundle:
                    cmds.append(cmd)
                    cmd = rsync_in
                    job_n = 0
    return cmds

#cmds = '\n'.join(cmds)
#swarm_file.write_text(cmds)

In [123]:
users=['nielsond']
user_cmds = {u:[] for u in users}
users = cycle(users)
pstep = 100
pranges = [(0,1001)]
i = 0
loop_n = 4
for pr in pranges:
    user = next(users)
    cmds = make_cmd_list(user, pr, bundle=1)
    jobs_per_swarm = 1001
    swarm_files = []
    for ii in range(0, len(cmds), jobs_per_swarm):
        swarm_file = Path('/data/MLDSST/abcd/abcd_swarm_files/swarm_test_crt_%d_%d_%s_%d_bad'%(pr[0],ii,user, loop_n))
        swarm_files.append(swarm_file.as_posix())
        swarm_file.write_text('\n'.join(cmds[ii:ii+jobs_per_swarm]))
        swarm_cmd = f'swarm -f {swarm_file} -g 12 -t auto --logdir ${{USER}}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"'
        user_cmds[user].append(swarm_cmd)
        print(len(cmds[ii:ii+jobs_per_swarm]))


7119

12
